# Counting Sort

A couple of nights ago I saw a video of JUG Milano about algorithms, the context was technical interviews. The video was the \#88 of JUG Milano.

During the speech I was intrigued about "continug sort": a way of sorting integers arrays in linear time, under some circumstances.

The idea is simple: in practice, with integers, is possible to "reconstruct" the ordered "array" using another array where indices represent the ordered array and content are the occurrences of an integer of the original array.

The I tried to write this in Python, and this is what came out.

In [1]:
def counting_sort(A):
    "Sort the 'array' of int A."
    if len(A) <= 1:
        return A
    M = A[0]
    m = A[0]
    for i in range(1, len(A)):
        if A[i] > M:
            M = A[i]
        if A[i] < m:
            m = A[i]
    # frequency array of the range
    C = [0] * (M - m + 1)
    for i in range(len(A)):
        C[A[i] - m] = C[A[i] - m] + 1
    # construct sorted A from C
    k = 0
    for i in range(len(C)):
        while C[i] > 0:
            A[k] = m + i
            C[i] = C[i] - 1
            k = k + 1
    return A

Some unit tests.

In [5]:
A = [3,2,5,6,3,8,9,11]
counting_sort(A)
assert A == [2,3,3,5,6,8,9,11]
A = [11,2]
counting_sort(A)
assert A == [2,11]
A = [2]
counting_sort(A)
assert A == [2]
A = [-1,-2,3,0]
counting_sort(A)
assert A == [-2,-1,0,3]
print('All good since then')

All good since then


Then I thought to post this code to the Python Milano Slack channel, my intent was to show how simple is to write algorithms in Python. It seems a sort of pseudocode for Counting Sort as expressed in "Introduction to algorithms" 3rd ed. Cormen, Leiserson, Rivest, Stein ch. 8.

But a nice discussion came out. "You can do it in a more pythonic way" the audience said. Ok, here is the revision.

In [6]:
def more_pythonic_counting_sort(A):
    "Sort the 'array' of int A."
    if len(A) <= 1:
        return A
    m = min(A)
    # frequency array of the range
    C = [0] * (max(A) - m + 1)
    for a in A:
        C[a - m] = C[a - m] + 1
    # construct sorted A from C
    k = 0
    for i in range(len(C)):
        while C[i] > 0:
            A[k] = m + i
            C[i] = C[i] - 1
            k = k + 1
    return A

Unit tests.

In [7]:
A = [3,2,5,6,3,8,9,11]
more_pythonic_counting_sort(A)
assert A == [2,3,3,5,6,8,9,11]
A = [11,2]
more_pythonic_counting_sort(A)
assert A == [2,11]
A = [2]
more_pythonic_counting_sort(A)
assert A == [2]
A = [-1,-2,3,0]
more_pythonic_counting_sort(A)
assert A == [-2,-1,0,3]
print('All good, again')

All good, again


The audience mostly agreed that this was the most pythonic achievable implementation, but a new "issue" came out. What is the performance of this stuff? So I started to write some performance tests.

In [14]:
import random

def test1():
    A = [random.randint(0, 99999) for i in range(100000)]
    counting_sort(A)

def test2():
    A = [random.randint(0, 99999) for i in range(100000)]
    more_pythonic_counting_sort(A)

def test3():
    A = [random.randint(0, 99999) for i in range(100000)]
    A.sort()

In [31]:
%time for i in range(10): test1()

CPU times: user 2.15 s, sys: 18.7 ms, total: 2.17 s
Wall time: 2.17 s


In [32]:
%time for i in range(10): test2()

CPU times: user 1.94 s, sys: 18.3 ms, total: 1.96 s
Wall time: 1.96 s


In [33]:
%time for i in range(10): test3()

CPU times: user 1.79 s, sys: 20.7 ms, total: 1.81 s
Wall time: 1.81 s


Ok, this demonstrates that writing pythonic code is worthy, the performance killer of the first implementation was the first for loop used to calculate the maximum and minimun of the range.

Surprisingly the pythonic version has good performance, near to the [Timsort](https://en.wikipedia.org/wiki/Timsort) implemented (in C) into the python interpreter itself, althougth Timsort is acting as a normal Mergesort in this case, because on average there are no pre sorted sequences in the randomic lists. Should be interesting to compare a Mergesort implementation written in python with Counting Sort.

Then I tried another tests. Use lists without repetitions.

In [34]:
def test4():
    # no repetitions
    A = list(range(100000))
    random.shuffle(A)
    more_pythonic_counting_sort(A)

def test5():
    # no repetitions
    A = list(range(100000))
    random.shuffle(A)
    A.sort()

In [35]:
%time for i in range(10): test4()

CPU times: user 1.43 s, sys: 11.9 ms, total: 1.44 s
Wall time: 1.44 s


In [36]:
%time for i in range(10): test5()

CPU times: user 1.25 s, sys: 12.1 ms, total: 1.26 s
Wall time: 1.26 s


In this case we are not so close, because the range is always 100000, before it can be K = M - m < 100000, so the counting list was smaller.

Ok, enough of messing up with ordering algorithms, I hope you enjoyed the discussion.